# 5. How To Read And Write To A Google Sheet With Python & Pandas

## Learning Outcomes

- To learn how to authenticate with Google Sheets within a .py file or jupyter notebook file.
- To learn how to read from a Google Sheet.
- To learn how to write (upload) to a Google Sheet.
- To learn how to maniuplate data within Google Sheets.
- To learn how to store data to a local file from a Google Sheet.
- To learn how to share a Google Sheet with other gmail users.

---------------------------------------------------------------

## Module Imports

In order to run the module imports below you will need to have installed the following python client:

- https://github.com/googleapis/google-api-python-client

A quick way to do that would be to run the following command and to restart your Jupyter Notebook server:

~~~

pip install --upgrade google-api-python-client

~~~

---

Additionally I would encourage you to download the gcloud software development kit (SDK), which will come in useful for deploying and access your Google Cloud Platform resources:

You can find more information on how to do that from:

- https://cloud.google.com/python/setup
- https://cloud.google.com/sdk/docs

In [195]:
pip install --upgrade google-api-python-client

Requirement already up-to-date: google-api-python-client in /opt/anaconda3/lib/python3.7/site-packages (1.8.4)
Note: you may need to restart the kernel to use updated packages.


In [196]:
import pandas as pd
import json
import csv
from google.oauth2 import service_account

----------------------------------------------------------------

## Download Pygsheets

We will be using a python package called [pygsheets](https://pygsheets.readthedocs.io/en/stable/) to easily interact with a new google sheet that we will create.

If you're using anaconda you can navigate to your terminal and install the latest version of this package with:

~~~

conda install pygsheets

~~~

Alternatively if you're using pip or pip3 to manage your packages you can type either:

~~~

pip install pygsheets
pip3 install pygsheets


~~~

In [197]:
!pip install pygsheets

As you can see from above I've previously installed pygsheets, after you've installed the python package if you're working from a jupyter notebook, then please restart the Kernel:

![how to clear the kernel](https://sempioneer.com/wp-content/uploads/2020/05/1_clear_kernel.png)

In [198]:
import pygsheets

--------------------------------------------------------------------------------------------------------------------------------

## To Authenticate To A Google Cloud Project With A .JSON Service Account Key

Using your local .json file we will authenticate to our google service account.

In [199]:
with open('service_account.json') as source:
    info = json.load(source)
    
credentials = service_account.Credentials.from_service_account_info(info)

--------------------------------------------------------

## Create A New Google Sheet + Obtain The Unique ID

Firstly we are going to create a new google sheet, then we'll obtain the id of that specific google sheet which can be found within the URL:

![how to obtain the google sheet id](https://sempioneer.com/wp-content/uploads/2020/05/2_obtain_the_google_sheet_id.png)

## Python With Google Sheet Commands

### Authenticating With Google Sheets With Pyghseets

Let's see how to successfully authenticate to google sheets with our .json key 

In [200]:
client = pygsheets.authorize(service_account_file='service_account.json')

---------------------------------------------------------------------------------

### How to connect to a specific google sheet

Now that we've authenticated pygsheets with our google cloud project let's connect to a specific google sheet. In order to do this we will need to grab the URL link and share the spreadsheet.

In [201]:
spreadsheet_url = \
"https://docs.google.com/spreadsheets/d/1OBR8HEE2rjaWqZPnb8mHnp3YpxxELcy5Zqve3gtX22Q/edit?usp=sharing"

Additionally we will extract the spreadsheet ID which can be located in the URL above between /d/ and /edit?

This id will be unique for you so please make sure to code along! 

![google spreadsheet](https://sempioneer.com/wp-content/uploads/2020/05/spreadsheet-id-.png)

In [202]:
sheet_data = client.sheet.get('1OBR8HEE2rjaWqZPnb8mHnp3YpxxELcy5Zqve3gtX22Q')

There are three ways to open the spreadsheet:

- By the spreadsheet title
- By the unique spreadsheet ID
- By the exact URL of the ghseet

Let's showcase all three and you can take your pick!

In [203]:
# sheet = client.open('Sheet1') ## You will need to activate the Google Drive API as well as the spreadsheet API for this one to work
sheet = client.open_by_key('1OBR8HEE2rjaWqZPnb8mHnp3YpxxELcy5Zqve3gtX22Q')
sheet = client.open_by_url('https://docs.google.com/spreadsheets/d/1OBR8HEE2rjaWqZPnb8mHnp3YpxxELcy5Zqve3gtX22Q/edit?usp=sharing')

In [204]:
print(sheet)

<Spreadsheet 'Test Google Sheet' Sheets:1>


------------------------------------------------------------------------------------------

### How To Select A Specific Google Worksheet

Let's select a worksheet was automatically created with the gsheet called Sheet1:

In [205]:
wks = sheet.worksheet_by_title('Sheet1')
print(wks)

<Worksheet 'Sheet1' index:0>


------------------------------------------------------------------------

### Accessing Rows & Columns Meta-data

After uploading a dataframe into a google sheet, you can see how many columns and rows you have with:
    
~~~

wks.cols # To view the number of columns
wks.rows # To view the number of rows

~~~

In [206]:
print(f"There are {wks.cols} columns in the gsheet!")
print(f"There are {wks.rows} rows in the gsheet!")

There are 31 columns in the gsheet!
There are 1001 rows in the gsheet!


-----------------------------------------------------------------

### How To Upload Data To A Google Sheet From A Pandas DataFrame

I am going to use the keyword data that I originally downloaded from Ahrefs, however if you have another csv that's absolutely fine. I'd encourage you to simply read any csv file and to create a pandas dataframe:

In [207]:
df = pd.read_csv('../1_Keywords/data/digital-marketing-keyword-ideas.csv', 
                 delimiter='\t', 
                 encoding='UTF-16')

df.drop(columns=['#'], inplace=True)

To upload a dataframe to this worksheet we can use the following syntax:
    
~~~

set_dataframe(df, start, copy_index=False, copy_head=True, extend=False, fit=False, escape_formulae=False, **kwargs)

~~~

In [208]:
wks.set_dataframe(df, start=(1,1))

![](https://sempioneer.com/wp-content/uploads/2020/05/python-pygsheets.png)

It's worth remembering that when you run the above command you will overwrite any cell values. Instead if you would like to progressively append rows within your google sheets please use the below syntax:

~~~
wks.set_dataframe(df, start=(1,1), extend=True)

~~~

--------------------------------------------------------------------------------------------------------

Pro Tip: If you don't have enough rows within your existing worksheet, then uploading a pandas dataframe might fail! In order to account for this you can check the existing number of rows within the worksheet vs the number of rows within the dataframe and add extra rows if required.

We can also do the same thing for columns too!

In [209]:
# If the number of rows within the worksheet is less than the dataframe:
if wks.rows < df.shape[0]:
    number_of_rows_to_add = df.shape[0] - wks.rows + 1 
    # Adding the required number of rows
    wks.add_rows(number_of_rows)
# If the number of cols within the worksheet is less than the dataframe:
elif wks.cols < df.shape[1]:
    number_of_cols_to_add = df.shape[1] - wks.cols + 1 
    wks.add_cols(number_of_cols_to_add)
else:
    pass

--------------------------------------------------------------------

### How To A Pandas Dataframe From A Specific Worksheet

We can also get data directly from the google sheets worksheet in the form of a dataframe:

~~~

get_as_df(has_header=True, index_column=None, start=None, end=None, numerize=True, empty_value='', value_render=<ValueRenderOption.FORMATTED_VALUE: 'FORMATTED_VALUE'>, **kwargs)[source]

~~~

In [210]:
dataframe_two = wks.get_as_df()

In [211]:
print(f"This new dataframe has {dataframe_two.shape[0]} rows and {dataframe_two.shape[1]} columns")

This new dataframe has 1000 rows and 11 columns


In [212]:
dataframe_two.head(6)

,Keyword,Country,Difficulty,Volume,CPC,Clicks,CPS,Return Rate,Parent Keyword,Last Update,SERP Features
0,coast,gb,42.0,70000.0,2.5,64715.0,0.93,1.67,coast,2020-05-09 23:55:39,Sitelinks
1,hubspot,gb,67.0,63000.0,5.0,59708.0,0.95,2.34,hubspot,2020-05-09 00:57:01,"Adwords top, Sitelinks, Top stories, Thumbnail..."
2,digital marketing,gb,74.0,19000.0,7.0,11033.0,0.57,1.41,digital marketing,2020-05-09 07:11:09,"Adwords top, Sitelinks, People also ask, Top s..."
3,digital,gb,89.0,16000.0,2.5,5912.0,0.38,1.19,digital,2020-05-09 02:26:10,"Sitelinks, People also ask, Top stories, Thumb..."
4,content meaning,gb,45.0,4400.0,17.0,622.0,0.14,1.18,content,2020-05-10 06:32:24,"Knowledge card, People also ask, Top stories, ..."
5,digital media,gb,24.0,3600.0,3.0,1671.0,0.47,1.24,digital media,2020-05-10 14:14:41,"Featured snippet, Thumbnails, People also ask,..."


------------------------------------------------------------------------------------------

### How To Get All Of The Google Sheet Values In A Python Format

We can also get all of the values in a list of lists:

In [213]:
all_values = wks.get_all_values()

In my case, notice how this has picked up all of the empty spaces that are located on the right hand side of the worksheet:

In [214]:
print(all_values[0])

['Keyword', 'Country', 'Difficulty', 'Volume', 'CPC', 'Clicks', 'CPS', 'Return Rate', 'Parent Keyword', 'Last Update', 'SERP Features', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']


A way to quickly remove the empty strings per list would be do a <strong> nested list comprehension: </strong>

In [215]:
cleaned_values = [[item for item in unique_list if item ]for unique_list in all_values]

------------------------------------------------

### How To Get Cell Ranges In Google Sheets With Python

You can also specific ranges similar to your excel functions for getting custom columns and rows.

In [216]:
cell_range = wks.range('A1:F10', 
          returnas='matrix')

In [217]:
print(cell_range)

[['Keyword', 'Country', 'Difficulty', 'Volume', 'CPC', 'Clicks'], ['coast', 'gb', '42', '70000', '2.5', '64715'], ['hubspot', 'gb', '67', '63000', '5', '59708'], ['digital marketing', 'gb', '74', '19000', '7', '11033'], ['digital', 'gb', '89', '16000', '2.5', '5912'], ['content meaning', 'gb', '45', '4400', '17', '622'], ['digital media', 'gb', '24', '3600', '3', '1671'], ['digital marketing agency', 'gb', '57', '3400', '18', '3925'], ['digital uk', 'gb', '24', '3100', '1.2', '2402'], ['what is digital marketing', 'gb', '74', '3100', '2.5', '2119']]


-----------------------------------------------------------------------------

### How To Get A Single Row For Extracting The Column Headers

We can get a single row with:

~~~ 

wks.get_row(row, returnas='matrix', include_tailing_empty=True, **kwargs)

~~~

In [218]:
headers = wks.get_row(1, include_tailing_empty=False)

In [219]:
print(headers)

['Keyword', 'Country', 'Difficulty', 'Volume', 'CPC', 'Clicks', 'CPS', 'Return Rate', 'Parent Keyword', 'Last Update', 'SERP Features']


Also notice how we've changed the include_tailing_empty to False which has automatically removed any empty strings in that row.

------------------------------------------------------------------------------------

### How To Extract A Single Column

Sometimes you might want to select a specific column from your worksheet:

In [220]:
first_column = wks.get_col(1)
first_column_data = first_column[1:] # We are doing a slide here to remove the header (keyword)

------------------------------------------------------------------------------------

### How To Sort By A Column

You can easily sort the google sheet by a specific column with:
    
~~~

wks.sort_range(start, end, basecolumnindex=0, sortorder='ASCENDING')
# Sorts the data in rows based on the given column index.


~~~

In [221]:
wks.sort_range(start='A2', end='L1001',basecolumnindex=4, sortorder='DESCENDING' )

<strong> Its worth knowing that np.nans (not a number) </strong> are treated as large numbers within google sheets. Therefore let's clear the sheet, remove all of the nans before uploading the dataframe, then we will perform this operation again.

![](https://sempioneer.com/wp-content/uploads/2020/05/python-pygsheets-nans.png)

------------------------------------------------------------------------------------

### How To Clear A Google Sheet With Python

You can clear all of the existing rows and columns within the sheet by referencing the same workspace variable (wks) and the .clear() syntax: 

~~~

wks.clear()

~~~

In [222]:
wks.clear()

----------------------------------------------------------------------

Now I'd recommend removing any nans before uploading your dataframe! So let's remove any np.nans within the Volume Column:

In [223]:
df.dropna(subset=['Volume'], inplace=True)
wks.set_dataframe(df, start=(1,1))

Let's try sorting again by the Volume column with pygsheets:

In [224]:
wks.sort_range(start='A2', end='L1001',basecolumnindex=4, sortorder='DESCENDING' )

![](https://sempioneer.com/wp-content/uploads/2020/05/sorted_volume.png)

-----------------------------------------------------------------------------

### How To Loop Over Every Row

You can iterate over row in your google sheet with a for loop:

In [225]:
for row in wks:
    print(row)

['Keyword', 'Country', 'Difficulty', 'Volume', 'CPC', 'Clicks', 'CPS', 'Return Rate', 'Parent Keyword', 'Last Update', 'SERP Features', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
['digital marketer', 'gb', '73', '1500', 'NaN', '1096', '0.72', '1.31', 'digital marketer', '2020-05-05 15:13:56', 'Sitelinks, People also ask, Top stories, Thumbnails, Adwords bottom', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
['detailed online', 'gb', '0', '450', 'NaN', '391', '0.91', '1.62', 'detailed online', '2020-05-03 13:55:48', 'Sitelinks, Top stories, Thumbnails', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
['what is this?', 'gb', '74', '300', 'NaN', '122', '0.43', '1.12', 'what is this', '2020-04-29 07:44:47', 'Sitelinks, Top stories, Thumbnails, Image pack', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
['define constitutes', 'gb', '8', '300', 'NaN', '23', 

--------------------------------------------------------------------------------

## Updating Specific Parts Of Your Google Sheet

Updating parts of your Google sheet can give you the flexibility to do a raw data dump into google sheets and then if other situations or API's change, you can specifically update any column, row or a range within your google sheet. Let's start by looking at how we can replace specific values:

### How To Replace Specific Values Within Your Google Sheet

Replacing values within your google sheets is incredibly easy and can be done via: 
    
~~~

wks.replace(pattern, replacement=None, **kwargs)

~~~

Let's for example try replacing any np.nans (not a number) in the Clicks and CPS column:

In [226]:
wks.replace("NaN", replacement="0")

![](https://sempioneer.com/wp-content/uploads/2020/05/replacing-nan-values.png)

Although this has indeed replaced the "NaN" values, it has done across all columns and rows within the worksheet. This might not be exactly what you're looking for.

<strong> For example maybe you're simply looking to replace cell values within 2 columns? </strong>

--------------------------------------------------------------------------------------------------------------------------------

### Introducing Find And Update Values

In [252]:
wks.clear()
wks.set_dataframe(df, start=(1,1))

We will find all of the cells between column 3 and 6 (exclusive so only two columns will be selected) that match against NaN (not a number):

In [228]:
cells = wks.find("NaN", searchByRegex=False, matchCase=False, 
     matchEntireCell=False, includeFormulas=False, 
    cols=(3,6), rows=None, forceFetch=True)

In [229]:
print(cells[0:5])

[<Cell E14 'NaN'>, <Cell E48 'NaN'>, <Cell F57 'NaN'>, <Cell F64 'NaN'>, <Cell E65 'NaN'>]


Then we will assign the value "Other" to all of the cells and update the values:

----------------------------------------------------------------------------------------------------

### How To Update A Range of Values

If you'd like to update a range of values within your google sheet you can use:

~~~
wks.update_values(crange=None, values=None, cell_list=None, extend=False, majordim='ROWS', parse=None)

~~~

In [138]:
for cell in cells:
    cell.value = "Other"
wks.update_values(cell_list=cells)

![](https://sempioneer.com/wp-content/uploads/2020/05/updating-specific-cells.png)

As you can see, we've only updated the CPC and Clicks column "NaN" values with "Other" and the other column values remain the same! 

In the example above we've specifically referenced a cell_list, however you could also use the two methods below:

~~~

# Update a single cell.
wks.update_value('A1', "Numbers on Stuff")

# Update the worksheet with the numpy array values. Beginning at cell 'A2'.
wks.update_values('A2', my_numpy_array.to_list())

~~~

--------------------------------------------------------------------------------

## Column Data Manipulation

### How To Add Multiple Columns

To add columns to your existing worksheet you can use this syntax:
    
~~~

wks.add_cols(3)

~~~

In [253]:
wks.add_cols(5)

![](https://sempioneer.com/wp-content/uploads/2020/05/how-to-add-extra-columns-with-python-pygsheets.png)

### How To Get The Column Names With The Column Index Positions

Firstly we will extract all of the headers on row 1, then we will wrap this variable with enumerate and convert it into a python dictionary. This will allow us to create a lookup table for index positions!

Notice below how we have also changed:

~~~

include_tailing_empty=True

~~~

The reason why is that we just want to make sure that if we had empty column values between tables, we would also pick up any empty string values between columns.

In [254]:
headers = wks.get_row(1, include_tailing_empty=True)
enumerated_headers = list(enumerate(headers))

In [255]:
print(f"{enumerated_headers}")

[(0, 'Keyword'), (1, 'Country'), (2, 'Difficulty'), (3, 'Volume'), (4, 'CPC'), (5, 'Clicks'), (6, 'CPS'), (7, 'Return Rate'), (8, 'Parent Keyword'), (9, 'Last Update'), (10, 'SERP Features'), (11, ''), (12, ''), (13, ''), (14, '')]


Now let's remove any of the enumerated headers that have an empty string with a list comprehension:

In [256]:
enumerated_headers = [tuple_object for tuple_object in enumerated_headers if tuple_object[1]]

In [257]:
print(f"These are the cleaned, enumerated headers: \n \n {enumerated_headers}")

These are the cleaned, enumerated headers: 
 
 [(0, 'Keyword'), (1, 'Country'), (2, 'Difficulty'), (3, 'Volume'), (4, 'CPC'), (5, 'Clicks'), (6, 'CPS'), (7, 'Return Rate'), (8, 'Parent Keyword'), (9, 'Last Update'), (10, 'SERP Features')]


-------------------------------------------------------------------------------------------

We can then convert this list of tuples into a python dictionary to create a look up table for columns versus their index positions:

In [258]:
lookup_table = dict(enumerated_headers)
lookup_table_reversed = {value: key for key, value in lookup_table.items()}

In [259]:
print(f"From this you can see how we've created a lookup table via a python dictionary where the key is the column name and the value is the index position: \n\n {lookup_table_reversed}")

From this you can see how we've created a lookup table via a python dictionary where the key is the column name and the value is the index position: 

 {'Keyword': 0, 'Country': 1, 'Difficulty': 2, 'Volume': 3, 'CPC': 4, 'Clicks': 5, 'CPS': 6, 'Return Rate': 7, 'Parent Keyword': 8, 'Last Update': 9, 'SERP Features': 10}


--------------------------------------------------------------------------------------------

### How To Delete Columns

You will need to pass two parameters when deleting columns via pyghseets: 

~~~

wks.delete_cols(index, number=1)

index – Index of first column to delete
number – Number of columns to delete


~~~

In [262]:
print(f"Currently we have {wks.cols} columns in the google sheet.")

Currently we have 15 columns in the google sheet.


In [263]:
max(lookup_table.keys()) + 1 

11

As we can see the highest index position that we have as a true column from row 1 is 10. Therefore let's delete all of the columns from index position 12 and greater:

Pro tip: Now the reason why it is 12 and not 11 is because pygsheets starts its index positioning for columns and rows at 1, whilst python starts its indexing at position 0. Also pygsheets is inclusive with the starting index (1 means from 1 onwards, rather than column 2 onwards).

In [264]:
wks.delete_cols(index=12,number=wks.cols - 10)

----------------------------------------------------------------

You can now easily delete multiple columns by creating your own column index with enumerate!

![](https://sempioneer.com/wp-content/uploads/2020/05/column-reduction.png)

------------------------------------------------------------------------

---------------------------------------------------------------

### How To Insert Multiple Columns

The following syntax allows you to insert multiple columns:
    
~~~

wks.insert_cols(col, number=1, values=None, inherit=False)


Parameters:
col – Index of the col at which the values will be inserted.
number – Number of columns to be inserted.
values – Content to be inserted into new columns.
inherit – New cells will inherit properties from the column to the left (True) or to the right (False).

~~~

---------------------------------------------------------------------------------

### How To Update A Single Column

In order to update a single column, use this syntax:
    
~~~

wks.update_col(index, values, row_offset=0)

~~~

We can use the previous lookup table that we created to search for a specific column:

In [271]:
wks.update_col(index = lookup_table_reversed['CPS']
               ,values = wks.rows * ["NA"], 
                row_offset=1)

['NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',

------------------------------------------------------------------------

### How To Adjust Column Width

--------------------------------------------------------

## Row Data Manipulation

### How To Insert Multiple Rows

### How To Insert A Column

----------------------------------------------------------------------

### How To Insert A Row

In [ ]:
add_rows(rows)

---------------------------------------------------------------

### How To Delete A Single Row

----------------------------------------------------------------------

### How To Delete A Single Column

---------------------------------------------------------------------------------------------------

### Formatting Google Sheets In Python

#### Frozen Rows + Columns

You can easily set the number of Frozen Rows + Columns with the two commands:
    
~~~

wks.frozen_rows(5)
wks.frozen_cols(3)
~~~


#### How To Bold Cells

----------------------------------------------------------------------------------------------------------------

#### Dropdown Boxes

------------------------------------------------------------------------

#### Horozintal Alignment

------------------------------------------------------------------------------------

### How to create a worksheet

------------------------------------------------------------------------------------------

### How to delete a worksheet

------------------------------------------------------------------------------------------

## Batching Your API Calls

In [ ]:
If you are going to make a lot of requests to the Google Sheets API, its recommended to batch your API requests. This ensures that you will 

---------------------------------------------------------------------------------------------------

### How To Share A Google Sheet With An Email Address In Python

You can easily share any connected spreadsheet with:
    
~~~

.share(email_or_domain, role='reader', type='user', **kwargs)

~~~

Additionally you can include an email message to your user!

In [36]:
spreadsheet.share('example@gmail.com', role='commenter', 
                  type='user', 
                  emailMessage='Here is the spreadsheet we talked about!')

------------------------------------------------------------------------

-------------------------------------------------------------------------------------

### How To Export A Google Sheet To A .CSV

Also its possible to easily export your google sheet to a .csv file with: 

~~~

wks.export(file_format=<ExportType.CSV: 'text/csv:.csv'>, filename=None, path='')

~~~

However <strong> we'll need to enable the Google Drive API </strong> for this to work, so quickly go to your Google Cloud Project and activate this API for the following command to work:

![](https://sempioneer.com/wp-content/uploads/2020/05/googel-drive-api.png)

In [117]:
wks.export(filename='this_is_a_csv_file')

----------------------------------------------------------------

### How To Convert A Google Sheet Into JSON

Sometimes when communicating with another developer, they would like to receive your data in the form of JSON. You can easily turn your google sheets data into a JSON API to be fed directly into another RESTI API by chaining multiple commands:
    
~~~

wks.get_as_df().to_json()

~~~

This will do two things:
- Firstly we will obtain the entire worksheet as a pandas dataframe.
- Secondly we'll convert the pandas dataframe into a JSON object.

In [39]:
wks.get_as_df().to_json()

'{"Keyword":{"0":"digital marketer","1":"detailed online","2":"what is this?","3":"define constitutes","4":"digital marketing agencies london","5":"services meaning","6":"marketeer meaning","7":"regardless marketing","8":"media channels","9":"on-digital","10":"the who.com","11":"hubspo","12":"marketer meaning","13":"digital marketing services birmingham","14":"who is this?","15":"digitals","16":"according to meaning","17":"who\'s definition","18":"why is digital marketing important","19":"huspot","20":"content marketing companies","21":"meaning of create","22":"like digital","23":"define services","24":"business define","25":"key-digital","26":"all digital","27":"define in fact","28":"what does defined mean","29":"trends definition in business","30":"what is for","31":"digital marketing campaigns","32":"hubstop","33":"who what why","34":"lucy definition","35":"digital marketing theory","36":"internet marketing services uk","37":"lucy alexander age","38":"what does a digital marketer do

---------------------------------------------